In [7]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [8]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [12]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


In [24]:
for i in range(5):
    print("Our current number is {}, the next is {}".format(i, i+1))

Our current number is 0, the next is 1
Our current number is 1, the next is 2
Our current number is 2, the next is 3
Our current number is 3, the next is 4
Our current number is 4, the next is 5


## 1.2 What happens if there's an error in our query?

In [13]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2028')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2028'

In [14]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [ ]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [15]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [18]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2028R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2380\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2010\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 651\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 191"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [37]:
def get_grades(grades):
    result = pd.DataFrame()
    
    for grade in grades: 
        example_naep_query = (
            'https://www.nationsreportcard.gov/'
            'Dataservice/GetAdhocData.aspx?'
            f'type=data&subject=writing&grade={grade}&'
            'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        naep_resp = requests.get(example_naep_query)
        
        try: 
            naep_resp_j = naep_resp.json()
            naep_resp_j
            naep_resp_d = pd.DataFrame(naep_resp_j['result'])
            result = pd.concat([result,naep_resp_d]) 
        
        except Exception as e:
            print(e)
    return result

In [40]:
# your code here

grades = ['4','8','12']

R = []

for grade in grades: 
    example_naep_query = (
        'https://www.nationsreportcard.gov/'
        'Dataservice/GetAdhocData.aspx?'
        f'type=data&subject=writing&grade={grade}&'
        'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
    naep_resp = requests.get(example_naep_query)
    
    try: 
        naep_resp_j = naep_resp.json()
        naep_resp_j
        naep_resp_d = pd.DataFrame(naep_resp_j['result'])
        # result = pd.concat([result,naep_resp_d]) 
        R.append(naep_resp_d)
    
    except Exception as e:
        print(e)
        
pd.concat(R)

Invalid control character at: line 1 column 289 (char 288)


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 3,
   'CohortLabel': 'Grade 12',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 12,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 141.256977963264,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 3,
   'CohortLabel': 'Grade 12',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 12,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 155.385916780351,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [41]:
## get the key
API_KEY = "1Gh9eKod5_-vX1otu_KEzcFphmJC58Kwb2qsJbcKPn76WUI7eW6biYfFe6R6Pmx_7NRaaQraF-Co_eLwcjvLl-DGQ4A0wHE8Z6WlFm9Bum6lEFvh9wiLufavouMtZXYx"

In [42]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [43]:
yelp_genjson['businesses'][0]

{'id': 'XVGEEIH5rVB2QzW-qywcJw',
 'alias': 'base-camp-cafe-hanover',
 'name': 'Base Camp Cafe',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 252,
 'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
 'rating': 4.4,
 'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3 Lebanon St',
  'address2': 'Ste 13',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
 'phone': '+16036432007',
 'display_phone': '(603) 643-2007',
 'distance': 196.1397581876442}

In [44]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'XVGEEIH5rVB2QzW-qywcJw',
 'alias': 'base-camp-cafe-hanover',
 'name': 'Base Camp Cafe',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 252,
 'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
 'rating': 4.4,
 'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3 Lebanon St',
  'address2': 'Ste 13',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
 'phone': '+16036432007',
 'display_phone': '(603) 643-2007',
 'distance': 196.1397581876442}

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,252,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758
1,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,545,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160
2,qxm3VNmD0O2zw8m9U8SxZg,duende-hanover-2,Duende,https://s3-media4.fl.yelpcdn.com/bphoto/rf_h2D...,False,https://www.yelp.com/biz/duende-hanover-2?adju...,15,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",3.7,"{'latitude': 43.70074272457673, 'longitude': -...",[],NaN,"{'address1': '15 Lebanon St', 'address2': '', ...",+16033064826,(603) 306-4826,107.388862
3,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,29,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.1,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552
4,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,False,https://www.yelp.com/biz/bistro-at-six-hanover...,2,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 43.7001146, 'longitude': -72.2877...",[],$$,"{'address1': '6 E South St', 'address2': None,...",+16036430600,(603) 643-0600,198.651788


In [45]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
qxm3VNmD0O2zw8m9U8SxZg,qxm3VNmD0O2zw8m9U8SxZg,duende-hanover-2,Duende,https://s3-media4.fl.yelpcdn.com/bphoto/rf_h2D...,https://www.yelp.com/biz/duende-hanover-2?adju...,NaN,+16033064826,(603) 306-4826
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
wyV_NfYn4ZOfp_sHMDPcAw,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,https://www.yelp.com/biz/bistro-at-six-hanover...,$$,+16036430600,(603) 643-0600
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/c2SgUR...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000


In [59]:
# change location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"

my_location_tpe = "Taipei,Taiwan,03456"
yelp_genquery_tpe = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location_tpe)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_tpe = requests.get(yelp_genquery_tpe, headers = header)
yelp_genresp_tpe

yelp_genresp_tpe.json()

<Response [200]>

{'businesses': [{'id': '5gNHAIyajmJ_4i5vXwOVRw',
   'alias': '永和豆漿大王-大安區-2',
   'name': 'Yong He Soy Milk King',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/g_CeWw_0dm3nvVUP8QPInA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/%E6%B0%B8%E5%92%8C%E8%B1%86%E6%BC%BF%E5%A4%A7%E7%8E%8B-%E5%A4%A7%E5%AE%89%E5%8D%80-2?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'review_count': 228,
   'categories': [{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
    {'alias': 'gourmet', 'title': 'Specialty Food'},
    {'alias': 'taiwanese', 'title': 'Taiwanese'}],
   'rating': 4.4,
   'coordinates': {'latitude': 25.029722, 'longitude': 121.543248},
   'transactions': [],
   'price': '$',
   'location': {'address1': '復興南路二段102號',
    'address2': '1樓',
    'address3': None,
    'city': "Da'an District",
    'zip_code': '106',
    'country': 'TW',
    'state': 'TPE',
    'di

In [60]:
yelp_genjson_tpe = yelp_genresp_tpe.json()

## turn JSON into usable data (DF)
yelp_gendf = pd.DataFrame(yelp_genjson_tpe['businesses'])
#list(yelp_gendf_marlow) # list columns
yelp_gendf['location1'] = yelp_gendf.location.apply(lambda loclist: loclist['address1'])
yelp_gendf[['alias', 'name', 'url', 'review_count', 'rating', 'location1', 'price']]

,alias,name,url,review_count,rating,location1,price
0,永和豆漿大王-大安區-2,Yong He Soy Milk King,https://www.yelp.com/biz/%E6%B0%B8%E5%92%8C%E8...,228,4.4,復興南路二段102號,$
1,上引水產-中山區,Addiction Aquatic Development,https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6...,414,4.4,民族東路410巷2弄18號,$$
2,阜杭豆漿-中正區,Fuhang Soy Milk,https://www.yelp.com/biz/%E9%98%9C%E6%9D%AD%E8...,399,4.2,忠孝東路一段108號,$
3,金峰魯肉飯-中正區,Jin Feng Braised Pork Rice,https://www.yelp.com/biz/%E9%87%91%E5%B3%B0%E9...,151,4.3,羅斯福路一段10-1號,$
4,原西園橋下-萬華區,原西園橋下,https://www.yelp.com/biz/%E5%8E%9F%E8%A5%BF%E5...,5,4.8,西園路一段242號,NaN
5,麥而美早餐店-台北市萬華區,麥而美早餐店,https://www.yelp.com/biz/%E9%BA%A5%E8%80%8C%E7...,3,4.7,成都路119號,NaN
6,西門金峰-台北市萬華區,Xi Men Jin Feng,https://www.yelp.com/biz/%E8%A5%BF%E9%96%80%E9...,9,4.2,昆明街89號,$
7,無老鍋-中山區,Wulao,https://www.yelp.com/biz/%E7%84%A1%E8%80%81%E9...,19,4.7,中山北路二段36-1號,$$
8,馬辣-萬華區-2,Mala Spicy Hot Pot,https://www.yelp.com/biz/%E9%A6%AC%E8%BE%A3-%E...,54,4.5,西寧南路157號,$$
9,萬華陳記專業腸蚵麵線-萬華區,萬華陳記專業腸蚵麵線,https://www.yelp.com/biz/%E8%90%AC%E8%8F%AF%E9...,14,4.5,和平西路三段166號,$


In [62]:
# change location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"

my_location_tpe = "Pacific Palisades, 90272"
yelp_genquery_tpe = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location_tpe)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_tpe = requests.get(yelp_genquery_tpe, headers = header)
# yelp_genresp_tpe

# yelp_genresp_tpe.json()

yelp_genjson_tpe = yelp_genresp_tpe.json()

## turn JSON into usable data (DF)
yelp_gendf = pd.DataFrame(yelp_genjson_tpe['businesses'])
#list(yelp_gendf_marlow) # list columns
yelp_gendf['location1'] = yelp_gendf.location.apply(lambda loclist: loclist['address1'])
yelp_gendf[['alias', 'name', 'url', 'review_count', 'rating', 'location1', 'price']]

,alias,name,url,review_count,rating,location1,price
0,cafe-vida-pacific-palisades-2,Cafe Vida,https://www.yelp.com/biz/cafe-vida-pacific-pal...,844,4.4,15317 Antioch St,$$
1,armav-los-angeles,Armav,https://www.yelp.com/biz/armav-los-angeles?adj...,81,4.7,970 Monument St,NaN
2,the-draycott-pacific-palisades,The Draycott,https://www.yelp.com/biz/the-draycott-pacific-...,542,4.0,15255 Palisades Village Ln,$$$
3,palisades-garden-cafe-pacific-palisades,Palisades Garden Cafe,https://www.yelp.com/biz/palisades-garden-cafe...,300,4.2,15231 La Cruz Dr,$
4,flour-cafe-and-pizzeria-pacific-palisades-2,Flour Cafe & Pizzeria,https://www.yelp.com/biz/flour-cafe-and-pizzer...,92,4.8,847 Vía De La Paz,NaN
5,gladstones-pacific-palisades-4,Gladstones,https://www.yelp.com/biz/gladstones-pacific-pa...,2424,3.0,17300 Pacific Coast Hwy,$$$$
6,porta-via-palisades-los-angeles,Porta Via Palisades,https://www.yelp.com/biz/porta-via-palisades-l...,186,3.5,1063 N Swarthmore Ave,$$
7,hanks-los-angeles-2,Hank's,https://www.yelp.com/biz/hanks-los-angeles-2?a...,318,3.7,1033 N Swarthmore Ave,$$
8,matthews-garden-cafe-pacific-palisades,Matthew's Garden Cafe,https://www.yelp.com/biz/matthews-garden-cafe-...,367,3.9,859 1/2 Swarthmore Ave,$$
9,sunset-smash-pacific-palisades,Sunset Smash,https://www.yelp.com/biz/sunset-smash-pacific-...,17,4.7,15120 Sunset Blvd,NaN


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [63]:
# your code here
biz_id = "Xbmdhb5PEfbAPNxzghWCRQ"


In [64]:
yelp_genquery_reviews

'https://api.yelp.com/v3/businesses/Xbmdhb5PEfbAPNxzghWCRQ/reviews'

In [65]:
# look at reviews of business with this id
base_url_reviews = f'https://api.yelp.com/v3/businesses/{biz_id}/reviews'
yelp_genquery_reviews = (base_url_reviews)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_reviews = requests.get(yelp_genquery_reviews, headers = header)
yelp_genresp_reviews

## then, look at structure of response
yelp_genjson_reviews = yelp_genresp_reviews.json()
yelp_genjson_reviews

<Response [200]>

{'reviews': [{'id': 'tDlXwQOhkgVBnK77GkICkw',
   'url': 'https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6%B0%B4%E7%94%A2-%E4%B8%AD%E5%B1%B1%E5%8D%80?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=tDlXwQOhkgVBnK77GkICkw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'text': 'We went to the standing sushi restaurant and were seated quite quickly. My only complaint was that water dripped a few times onto our head or shoulders from...',
   'rating': 5,
   'time_created': '2024-09-14 22:09:44',
   'user': {'id': 'gjzBT6_LS2Yj2O-kE8opiw',
    'profile_url': 'https://www.yelp.com/user_details?userid=gjzBT6_LS2Yj2O-kE8opiw',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/LK62eeBtLnDm-7kd4qFh2w/o.jpg',
    'name': 'Jenny H.'}},
  {'id': 'skoAYgr6bvCjEpLkMARfKw',
   'url': 'https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6%B0%B4%E7%94%A2-%E4%B8%AD%E5%B1%B1%E5%8D%80?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=skoAYgr6bvCjEpLkMARfKw&utm_campaig